<h1> Eptiope Prediction </h1>
<h3> by Benjamin Schubert </h3>

This tutorial illustrates the use of Fred2 to predict HLA-I/II epitopes and how to analyse results. Fred2 offers a long list on epitope prediction methods and. It was also designed in such a way that extending Fred2 with your favourit method is easy.

This tutorial will entail:
- Simple epitope prediction from a list of peptide sequences and protein sequences
- Consensus prediction with multiple prediction methods
- Manipulation of the results
- Integration of a new prediction method

<h2> Chapter 1: The basics </h2>
<br/>
We first start with importing the needed packages.

In [1]:
#is is needed as long as Fred2 is not installed properly
import sys 
sys.path.append("/Users/schubert/Dropbox/PhD/Porgramming/Fred2/")

from Fred2.Core import Allele, Peptide, Protein
from Fred2.IO import read_lines, read_fasta
from Fred2.EpitopePrediction import EpitopePredictorFactory

Lets start of with something playn simple: Defining peptides, and HLA alleles. You find all basic classes under `Fred2.Core`. HLA alleles have to be defined in new nomenclature (see: <a href="http://hla.alleles.org/">http://hla.alleles.org/</a>). 

In [2]:
peptides = [Peptide("SYFPEITHI"),Peptide("FIASNGVKLV"), Peptide("LLGATCMFV")]
allele = Allele("HLA-A*02:01")

Of course we don't have to specify all peptides and alleles by hand. We can also read those from file with `read_lines`. For doing so, the file has to contain the peptide sequences and HLA alleles one per line. 

In [3]:
peptides2 = read_lines("../Data/examples/peptide.txt", "Peptide")
alleles = read_line("../Data/examples/alleles.txt","Allele")

IOError: [Errno 2] No such file or directory: '../Data/examples/peptide.txt'